# Implementation

The algorithm used by **_stl-decomp-4j_** takes advantage of the fact that the data that is being decomposed is regularly spaced with no missing values to implement the underlying Loess smoothers in an efficient fashion. This document walks through the formulation of local linear interpolation used by the Loess interpolator to explain the flavor of the implementation, and then extends this analysis to the quadratic interpolation that was added to the Java implementation.

## Local Linear Interpolation

This section walks through the derivation of the code used to perform local (weighted) linear interpolation in `LinearLoessInterpolator`, which is a straight port of the code in the original Ratfor function `stl.r:est`.

The input data is a sequence of data points ${x_i, y_i}$, where the $x_i$ are the regularly spaced grid points.

The goal is to derive a set of weights $w_i$ such that the Loess interpolation of the data set at an arbitrary point $x$ can be written as

\begin{equation*}
  y(x) = \sum_{i = 1}^m \hat{w}_i(x_i) y_i
\end{equation*}

i.e. the interpolation is re-cast as a linear operation on the input y-values.

We can write a measure of the square error from weighted least-squares interpolation as

\begin{equation*}
E = {\frac{1}{2}} \sum_{i=1}^m (y_i - \alpha - \beta x_i)^2 \cdot w_i
\end{equation*}

where $\sum w_i = 1$ are external weights (in Loess these come from the implementation of the locality window).

### Finding $\alpha$

The optimal choices of $\alpha$ and $\beta$ are found by differentiating with respect to these, setting to 0 and solving:

\begin{equation*}
\frac{\partial E}{\partial \alpha} = - \sum_{i=1}^m w_i (y_i - \alpha - \beta x_i) = 0
\end{equation*}

Then

\begin{equation*}
\sum w_i y_i - \alpha \sum w_i - \beta \sum w_i x_i = 0
\end{equation*}

Or, defining $\langle y \rangle = \sum w_i y_i$, $\langle x \rangle = \sum w_i x_i$ and recalling that the $w_i$ are normalized, this can be rewritten as:

\begin{equation*}
\alpha = \langle y \rangle - \beta \langle x \rangle
\end{equation*}

### Finding $\beta$

Repeating this exercise for $\beta$, skipping the intermediate details, gives

\begin{equation*}
\beta = \frac{\langle x y \rangle - \langle x \rangle \langle y \rangle}{\langle x^2 \rangle - \langle x \rangle^2}
\end{equation*}

### The weight

Given the expressions above, we can write our expression for the interpolant at a point $x_i$ as

\begin{align*}
y(x_i) &= \alpha + \beta x_i \\
       &= \langle y \rangle + \beta ( x_i - \langle x \rangle) \\
       &= \langle y \rangle + \frac{\langle x y \rangle - \langle x \rangle \langle y \rangle}{\langle x^2 \rangle - \langle x \rangle^2} (x_i - \langle x 
\end{align*}

Writing out the averages that involve $y$ we have:

\begin{align*}
y(x_i) &= \sum_j w_j y_j + \sum_j \frac{x_i - \langle x \rangle}{\langle x^2 \rangle - \langle x \rangle^2} w_j (y_j(x_j - \langle x \rangle)) \\
       &= \sum_j w_j \left[1 + \frac{x_i - \langle x \rangle}{\langle x^2 \rangle - \langle x \rangle^2}(x_j - \langle x \rangle) \right] y_j \\
       &= \sum_j \hat{w}_j y_j
\end{align*}

where we define:

\begin{equation*}
\hat{w}_j \equiv w_j\left[ 1 + \frac{(x_i - \langle x \rangle)(x_j - \langle x \rangle)}{\langle x^2 \rangle - \langle x \rangle^2} \right]
\end{equation*}

## Local Quadratic Interpolation

Now we model the data as a local quadratic:

\begin{equation*}
y_i = a_0 + a_1 x_i + a_2 x_i^2 + \epsilon_i
\end{equation*}

The goal is to, again, write this as a linear combination of the $y_i$ values

\begin{equation*}
y(x_i) = \sum_i w_i y_i
\end{equation*}

### $a_0$